In [2]:

import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns

import torch

import torch.nn as nn
import torch.nn.functional as F
# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
# from Corpus import Corpus
os.chdir(cwd)

from collections import Counter, defaultdict
import itertools


from tqdm.auto import tqdm, trange
from collections import Counter
import random
from torch import optim

from torch.utils.tensorboard import SummaryWriter

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")
import pickle as pkl
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import nltk
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate

torch.set_default_dtype(torch.float32)

suffix = "full"

/Users/stephentoner/miniconda3/envs/si699proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('sentence_embed.pkl', 'rb') as f:
    embed = pkl.load(f)

embed_df = pd.DataFrame(embed)

In [4]:
embed_df = embed_df.rename(columns = {0: 'seqid', 1: 'passage_key', 2: 'sent_embeddings'})

In [5]:
data = U.load_file('data_vFFFF.pkl', 'pkl', config['DATADIR'])

In [6]:
data_df = pd.DataFrame(data)
data_df.head()

,author_id,author_name,book_id,gutenbergbookid,title,text,text_lines,authoryearofbirth,authoryearofdeath,downloads,subjects,topic,Sub_A,Sub_B,Sub_C,str_text_lines,passage_key
0,2210,"Verne, Jules",PG103,PG103,Around the World in Eighty Days,"[“No.”, , “I will buy it of you.”, , “No.”, , ...",7736,1828.0,1905.0,2015.0,"{'Voyages around the world -- Fiction', 'Adven...",Fiction,Adventure stories,Voyages around the world,Fiction,7736,PG103_7736
1,2210,"Verne, Jules",PG103,PG103,Around the World in Eighty Days,"[Bombay, for which they were now detained at C...",3686,1828.0,1905.0,2015.0,"{'Voyages around the world -- Fiction', 'Adven...",Fiction,Adventure stories,Voyages around the world,Fiction,3686,PG103_3686
2,2210,"Verne, Jules",PG103,PG103,Around the World in Eighty Days,"[, “Well, Monsieur Fix,” said Passepartout, “h...",4386,1828.0,1905.0,2015.0,"{'Voyages around the world -- Fiction', 'Adven...",Fiction,Adventure stories,Voyages around the world,Fiction,4386,PG103_4386
3,2210,"Verne, Jules",PG103,PG103,Around the World in Eighty Days,"[but the intractable Fogg, as reserved as ever...",3836,1828.0,1905.0,2015.0,"{'Voyages around the world -- Fiction', 'Adven...",Fiction,Adventure stories,Voyages around the world,Fiction,3836,PG103_3836
4,2210,"Verne, Jules",PG103,PG103,Around the World in Eighty Days,"[“I am he.”, , “Is this man your servant?” add...",3536,1828.0,1905.0,2015.0,"{'Voyages around the world -- Fiction', 'Adven...",Fiction,Adventure stories,Voyages around the world,Fiction,3536,PG103_3536


In [7]:
embed_df = embed_df.merge(data_df, how= 'left', left_on= 'passage_key', right_on = 'passage_key')

In [8]:
embed_df.head()

,seqid,passage_key,sent_embeddings,author_id,author_name,book_id,gutenbergbookid,title,text,text_lines,authoryearofbirth,authoryearofdeath,downloads,subjects,topic,Sub_A,Sub_B,Sub_C,str_text_lines
0,0,PG105_2371,"[-0.40695864, 0.3818504, 0.060175154, -0.16981...",5049.0,"Austen, Jane",PG105,PG105,Persuasion,[Croft’s next words explained it to be Mr Went...,2371.0,1775.0,1817.0,2778.0,"{'Regency fiction', 'Ship captains -- Fiction'...",Fiction,Love stories,Young women,Psychological fiction,2371
1,1,PG105_2371,"[-0.33000952, 0.35164443, 0.05541102, -0.16663...",5049.0,"Austen, Jane",PG105,PG105,Persuasion,[Croft’s next words explained it to be Mr Went...,2371.0,1775.0,1817.0,2778.0,"{'Regency fiction', 'Ship captains -- Fiction'...",Fiction,Love stories,Young women,Psychological fiction,2371
2,2,PG105_2371,"[-0.28961048, 0.38467273, 0.06806766, -0.14219...",5049.0,"Austen, Jane",PG105,PG105,Persuasion,[Croft’s next words explained it to be Mr Went...,2371.0,1775.0,1817.0,2778.0,"{'Regency fiction', 'Ship captains -- Fiction'...",Fiction,Love stories,Young women,Psychological fiction,2371
3,3,PG105_2371,"[-0.510949, 0.36323312, 0.12687655, -0.1873095...",5049.0,"Austen, Jane",PG105,PG105,Persuasion,[Croft’s next words explained it to be Mr Went...,2371.0,1775.0,1817.0,2778.0,"{'Regency fiction', 'Ship captains -- Fiction'...",Fiction,Love stories,Young women,Psychological fiction,2371
4,4,PG105_2371,"[-0.4631606, 0.34937078, 0.07573335, -0.178711...",5049.0,"Austen, Jane",PG105,PG105,Persuasion,[Croft’s next words explained it to be Mr Went...,2371.0,1775.0,1817.0,2778.0,"{'Regency fiction', 'Ship captains -- Fiction'...",Fiction,Love stories,Young women,Psychological fiction,2371


In [9]:
embed_df = embed_df.dropna(subset=['author_id', 'sent_embeddings'])

In [10]:

n_classes = embed_df.author_id.nunique()

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# label_encoder=OneHotEncoder(sparse_output=False)
# label_encoder=OneHotEncoder()
label_encoder=LabelEncoder()


In [11]:
y= label_encoder.fit_transform(embed_df['author_id']) #.to_numpy(dtype='int32').reshape(-1,1)
# y = y.toarray()

# y = embed_df['author_id'].astype('category')
X = np.vstack(embed_df['sent_embeddings'])



In [48]:
type(y[0])

numpy.int64

In [12]:
test_size = 0.2
val_size = 0.2
random_state =699

X_train, X_test, y_train, y_test = U.train_test_split(X, y, test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=y)

# Split train set into train and validation sets
X_train, X_val, y_train, y_val = U.train_test_split(X_train, y_train, test_size=val_size/(1-test_size),
                                                    random_state=random_state,
                                                    stratify=y_train)

In [16]:
import pandas as pd
import numpy as np
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC



(441612, 100)

In [65]:
embed_df.columns

Index(['seqid', 'passage_key', 'sent_embeddings', 'author_id', 'author_name',
       'book_id', 'gutenbergbookid', 'title', 'text', 'text_lines',
       'authoryearofbirth', 'authoryearofdeath', 'downloads', 'subjects',
       'topic', 'Sub_A', 'Sub_B', 'Sub_C', 'str_text_lines'],
      dtype='object')

In [14]:
rf_base = RandomForestClassifier(random_state=699).fit(X_train, y_train)

train_precision, train_recall, _ = (
    U.precision_recall_curve(y_train, rf_base.predict_proba(X_train)[:, 1])
)
val_precision, val_recall, _ = (
    U.precision_recall_curve(y_val, rf_base.predict_proba(X_val)[:, 1])
)

train_auprc = U.auc(train_recall, train_precision)
val_auprc = U.auc(val_recall, val_precision)

print(round(train_auprc, 3))
print(round(val_auprc, 3))

KeyboardInterrupt: 

In [ ]:
device = 'cpu'
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if (device == "cuda:0" or device == 'mps') else {}
collate_func = lambda x: tuple(x_.to(device) for x_ in default_collate(x)) if device != "cpu" else default_collate(x)

In [17]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)


In [ ]:
class DocumentAttentionClassifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_size, num_heads, hidden_dim, embeddings_fname, n_classes):
        '''
        Creates the new classifier model. embeddings_fname is a string containing the
        filename with the saved pytorch parameters (the state dict) for the Embedding
        object that should be used to initialize this class's word Embedding parameters
        '''
        super(DocumentAttentionClassifier, self).__init__()
        
        # Save the input arguments to the state
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.num_heads = num_heads
        self.embeddings_fname = vocab_size        

        self.linear = nn.Linear(num_heads * embedding_size, n_classes)
        self.lstm = nn.LSTM(embedding_size, hidden_dim, batch_first=True, bidirectional=True)
        self.attention = torch.rand(self.num_heads, hidden_dim * 2, requires_grad = True, device=device)
        # self.attention = torch.rand(self.num_heads, self.embedding_size, requires_grad = True, device=device)
        
    def forward(self, w):
        w = w.squeeze(1)
        # w = torch.t(self.embeddings(word_ids).squeeze()) # Embedding_Dim 
        lstm_out, _ = self.lstm(w.T)
        r = torch.matmul(self.attention, lstm_out.T)
        a = torch.softmax(r, -1)
        reweighted = a @ w
        output = self.linear(reweighted.view(-1))

        return torch.softmax(output, dim=0), a.T

In [ ]:
datasets = {}

datasets['train'] = list(zip(X_train, y_train))
datasets['val'] = list(zip(X_val, y_val))
datasets['test'] = list(zip(X_test, y_test))

train_list = datasets['train']
val_list = datasets['val']
test_list = datasets['test']

model = DocumentAttentionClassifier(1, 100, 4, 32, 'trained_model_final', n_classes)

In [ ]:
def run_eval(model, eval_data, kwargs):
    '''
    Scores the model on the evaluation data and returns the F1
    Eval Data must be in DataLoader-ready format
    '''

    eval_loader = DataLoader(eval_data, batch_size = 1, shuffle = False, collate_fn=collate_func, **kwargs)

    threshold = 0.2
    probs  = np.zeros((len(eval_loader), n_classes))
    labels = []
    
    with torch.no_grad():
        for idx, x in enumerate(eval_loader):
            word_ids, label = x
            labels.append(label.cpu().numpy())
            word_ids = word_ids.unsqueeze(1)
            output, weights = model(word_ids)
            probs[idx] = output.cpu().numpy()
    
    
    y_pred = np.argmax(probs, 1)
    labels = np.array(labels).squeeze().argmax(1)
    
    return labels, y_pred, f1_score(labels, y_pred, average='weighted')

In [ ]:

loss_period = 5
# model = model.to(device)
writer = SummaryWriter()
loss_function = nn.CrossEntropyLoss()

# VVV GOLD STANDARD VVV
optimizer = optim.AdamW(model.parameters(), lr = 5e-3, weight_decay = 0.1)
# ^^^ GOLD STANDARD ^^^

# optimizer = optim.AdamW(model.parameters(), lr = 5e-3, weight_decay = 0.001)

# optimizer = optim.AdamW(model.parameters())
# optimizer = optim.RMSprop(model.parameters(), 5e-3)
# optimizer = optim.SGD(model.parameters(), lr = 5e-4)

train_loader = DataLoader(train_list, batch_size=16, shuffle=True, collate_fn=collate_func, **kwargs)
n_epochs = 10
n_epochs = 1

# + vscode={"languageId": "python"}
loss_idx = 0
loss_record = []
model.train()

# + vscode={"languageId": "python"}r
for epoch in tqdm(range(n_epochs)):

    loss_sum = 0

    for step, data in tqdm(enumerate(train_loader)):

        word_ids, labels = data
        labels = labels.argmax()
        model.train()
        model.zero_grad()
        output, weights = model(word_ids)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        
        # TODO: Based on the details in the Homework PDF, periodically
        # report the running-sum of the loss to tensorboard. Be sure
        # to reset the running sum after reporting it.

        if not step % loss_period and step:
            writer.add_scalar("Loss", loss_sum, loss_idx)
            # if not step % (loss_period * 10) and step:
            #     model.eval()
            #     _y, _y2, f1 = run_eval(model, dev_list, kwargs)
            #     writer.add_scalar("F1", f1, loss_idx)
            #     model.train()
            loss_record.append(loss_sum)
            loss_sum = 0
            loss_idx += 1
            

        # TODO: it can be helpful to add some early stopping here after
        # a fixed number of steps (e.g., if step > max_steps)
        

# once you finish training, it's good practice to switch to eval.
model.eval()

y_true, y_pred, f1 = run_eval(model, val_list, kwargs)
print("Eval F1 Score of : "+ str(f1))

In [ ]:
y_true, y_pred, f1 = run_eval(model, test_list, kwargs)
print("Test F1 Score of : "+ str(f1))

In [ ]:
torch.save(optimizer.state_dict(), 'trained_opt_' + suffix)
torch.save(model.state_dict(), 'trained_model_' + suffix)